In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Creating a recommender system that performs basic collaborative filtering

In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 117.5 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-macosx_10_9_x86_64.whl size=1148083 sha256=b91cbcf0da27fafeacfcaec61a2a6a27cbc1fab83581c28ec47da933b799f3c2
  Stored in directory: /Users/tarunvisvar/Library/Caches/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd


from surprise import Dataset
from surprise import Reader

file_path = 'core-data-train_rating.csv'
reader = Reader(line_format="user item rating", skip_lines=1, rating_scale=(1,5))
df = pd.read_csv(file_path)[:100000]
df.drop("dateLastModified", axis=1, inplace=True)
data = Dataset.load_from_df(df, reader=reader)

data

In [2]:
# using svd algorithm for collaborative filtering
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import SVD

algo = SVD()
trainset, testset = train_test_split(data, test_size=0.25)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.8272
MAE:  0.6070


0.607019171225771

In [3]:
def recipe_ID_to_Name():


    #Returns a dictionary containing recipe id and their corresponding name
  recipe_df = pd.read_csv('core-data_recipe.csv')
  recipe_id_name = dict()

  for index, row in recipe_df.iterrows():
    recipe_id = int(row['recipe_id'])
    recipe_name = row['recipe_name']
    recipe_id_name[recipe_id] = recipe_name 
    
  return recipe_id_name

In [4]:
recipe_id_name = recipe_ID_to_Name()

In [5]:
from collections import defaultdict

def get_top_n_predictions(predictions, top_n):
    #returns top_n predicitons from the predicitons
  top_n_predictions = defaultdict(list)

  for uid, iid, true_r, est, _ in predictions:
    top_n_predictions[uid].append((iid, est))
  
  for uid, user_ratings in top_n_predictions.items():
    user_ratings.sort(key = lambda x:x[1], reverse = True)
    top_n_predictions[uid] = user_ratings[:top_n]
  
  return top_n_predictions

In [6]:
top_n_predictions = get_top_n_predictions(predictions=predictions, top_n=3)
top_n_predictions

defaultdict(list,
            {290868: [(10687, 4.710372530553498),
              (78299, 4.466651754678757),
              (9599, 4.281708633588556)],
             3746251: [(18019, 5),
              (14231, 4.893113439506732),
              (23891, 4.88980546136261)],
             3350000: [(17036, 4.66113252632197), (14746, 4.552595090918045)],
             3466115: [(26692, 4.733181412273534),
              (8626, 4.658231631533645),
              (7245, 4.648893444216181)],
             1808791: [(76808, 5), (8932, 5), (9999, 5)],
             1704050: [(7238, 5),
              (10497, 4.949703854382932),
              (96749, 4.925024788425924)],
             1378010: [(85114, 4.778121981765609),
              (106859, 4.70200420638865),
              (99211, 4.629091187369338)],
             805149: [(25874, 5), (24457, 5), (6683, 5)],
             775284: [(8446, 5), (69919, 5), (56927, 5)],
             97390: [(11843, 4.786857181604975),
              (26740, 4.75248898556793

In [7]:
def print_top_n_predictions(top_n_predictions, n_users):
  count = 0

  for uid, user_ratings in top_n_predictions.items():
    print(uid, ",", [recipe_id_name[iid] for iid, est in user_ratings])

    count += 1
    if count == n_users:
      break

In [8]:
print_top_n_predictions(top_n_predictions, 10)

290868 , ["Mrs. Sigg's Snickerdoodles", 'Boilermaker Tailgate Chili', 'Quick and Easy Brownies']
3746251 , ['Pepperoni Bread', 'Guacamole', 'Grilled Cheese Sandwich']
3350000 , ['Pumpkin Pancakes', 'Mushroom Pork Chops']
3466115 , ["Annie's Fruit Salsa and Cinnamon Chips", 'Yummy Honey Chicken Kabobs', "Jay's Signature Pizza Crust"]
1808791 , ['Italian Sausage, Peppers, and Onions', 'Fruity Curry Chicken Salad', 'Peanut Butter Bars I']
1704050 , ['Popovers', "Beth's Spicy Oatmeal Raisin Cookies", 'Pretzel Turtles®']
1378010 , ['Bread in a Bag', 'Classic Tapioca Pudding', 'Perfect Sushi Rice']
805149 , ['Shrimp Scampi Bake', 'Coconut Rum Balls', "Grandma VanDoren's White Bread"]
775284 , ['Chocolate Cavity Maker Cake', 'Bacon Wrapped Smokies', 'Delicious Ham and Potato Soup']
97390 , ['Irish Chicken and Dumplings', 'Chocolate Chip Cheese Ball', 'Black Bean and Couscous Salad']


# calculating fsa score

In [11]:
import ast
def recipe_id_to_Nutritions():
    
    recipe = pd.read_csv('core-data_recipe.csv')
    recipeID_to_nutritions = dict()
    
    for index,row in recipe.iterrows():
        recipeID = int(row['recipe_id'])
        nutritions = ast.literal_eval(row['nutritions'])
        recipeID_to_nutritions[recipeID] = (nutritions['fat']['amount'],nutritions['saturatedFat']['amount'],nutritions['sugars']['amount'])
    return recipeID_to_nutritions

nutrition = recipe_id_to_Nutritions()
nutrition

{240488: (11.67521, 3.646474, 19.84146),
 218939: (23.62519, 5.683611, 0.2361299),
 87211: (29.36761, 10.91876, 1.97421),
 245714: (7.600288, 3.975159, 2.452803),
 218545: (7.977177, 3.887164, 8.222667),
 20453: (28.79442, 14.09626, 3.700523),
 244856: (8.858574, 2.248184, 0.8452163),
 22402: (6.319194, 2.333397, 26.45939),
 258163: (35.29628, 6.657913, 7.08219),
 23658: (9.979216, 3.895137, 6.171535),
 244786: (13.56927, 3.387094, 4.319087),
 260453: (22.6428, 4.325186, 17.9497),
 103305: (19.03824, 7.078082, 10.5931),
 26668: (21.87788, 5.713773, 7.555897),
 185816: (14.714, 5.082802, 28.73791),
 79814: (14.03037, 5.369272, 7.653133),
 25507: (4.138773, 2.114902, 0.3038419),
 238886: (35.49799, 6.706356, 6.8054),
 235633: (74.25414, 30.12737, 6.996001),
 50908: (19.84756, 5.398934, 4.618526),
 73910: (34.56888, 16.41636, 0.1721675),
 236920: (16.67211, 6.121202, 22.6624),
 235757: (27.6036, 10.06171, 2.596323),
 8778: (61.71077, 37.1624, 6.755114),
 217969: (21.07427, 3.831406, 2.667

In [12]:
recipe_id_nutritions = recipe_id_to_Nutritions()

In [13]:
recipe_id_nutritions

{240488: (11.67521, 3.646474, 19.84146),
 218939: (23.62519, 5.683611, 0.2361299),
 87211: (29.36761, 10.91876, 1.97421),
 245714: (7.600288, 3.975159, 2.452803),
 218545: (7.977177, 3.887164, 8.222667),
 20453: (28.79442, 14.09626, 3.700523),
 244856: (8.858574, 2.248184, 0.8452163),
 22402: (6.319194, 2.333397, 26.45939),
 258163: (35.29628, 6.657913, 7.08219),
 23658: (9.979216, 3.895137, 6.171535),
 244786: (13.56927, 3.387094, 4.319087),
 260453: (22.6428, 4.325186, 17.9497),
 103305: (19.03824, 7.078082, 10.5931),
 26668: (21.87788, 5.713773, 7.555897),
 185816: (14.714, 5.082802, 28.73791),
 79814: (14.03037, 5.369272, 7.653133),
 25507: (4.138773, 2.114902, 0.3038419),
 238886: (35.49799, 6.706356, 6.8054),
 235633: (74.25414, 30.12737, 6.996001),
 50908: (19.84756, 5.398934, 4.618526),
 73910: (34.56888, 16.41636, 0.1721675),
 236920: (16.67211, 6.121202, 22.6624),
 235757: (27.6036, 10.06171, 2.596323),
 8778: (61.71077, 37.1624, 6.755114),
 217969: (21.07427, 3.831406, 2.667

In [14]:
nutrient_limits = {'fat' : (3, 17.5), 'saturatedFat' : (1.5, 5), 'sugars' : (5, 22.5)}

In [15]:
limits = {'fat':(3,17.5),'saturatedFat':(1.5,5),'sugars':(5,22.5)}
#Above data taken from traffic light signalling wikipedia page : https://en.wikipedia.org/wiki/Traffic_light_rating_system
def calculate_fsa(nutrient,amount):
    if amount<=limits[nutrient][0]:return 1
    elif amount<=limits[nutrient][1]:return 2
    else:return 3


In [16]:
def calculate_recipe_fsa(recipe_id):
    fat,saturatedFat,sugars = nutrition[recipe_id]
    return calculate_fsa('fat',fat)+calculate_fsa('saturatedFat',saturatedFat)+calculate_fsa('sugars',sugars)

In [17]:
recipe_fsa = dict()
for recipe_id in recipe_id_name: 
  recipe_fsa[recipe_id] = calculate_recipe_fsa(recipe_id)

In [18]:
recipe_fsa

{240488: 6,
 218939: 7,
 87211: 7,
 245714: 5,
 218545: 6,
 20453: 7,
 244856: 5,
 22402: 7,
 258163: 8,
 23658: 6,
 244786: 5,
 260453: 7,
 103305: 8,
 26668: 8,
 185816: 8,
 79814: 7,
 25507: 5,
 238886: 8,
 235633: 8,
 50908: 7,
 73910: 7,
 236920: 8,
 235757: 7,
 8778: 8,
 217969: 6,
 233910: 7,
 231459: 5,
 233874: 9,
 133604: 5,
 55420: 4,
 231825: 7,
 142456: 8,
 140407: 5,
 26623: 7,
 22681: 6,
 99892: 6,
 219884: 7,
 25930: 7,
 241535: 5,
 239547: 5,
 244979: 8,
 219754: 7,
 20900: 7,
 125366: 8,
 23613: 6,
 63351: 7,
 16246: 8,
 152220: 5,
 212912: 8,
 213650: 6,
 17178: 5,
 130444: 7,
 231412: 8,
 233443: 7,
 8987: 6,
 191159: 7,
 229331: 6,
 53650: 7,
 214631: 7,
 8657: 7,
 216704: 7,
 236517: 5,
 8733: 5,
 214755: 5,
 215053: 7,
 9035: 9,
 237375: 7,
 31870: 7,
 12414: 7,
 216090: 8,
 57002: 7,
 213268: 7,
 230270: 8,
 28011: 5,
 245367: 7,
 238325: 7,
 219208: 6,
 85757: 7,
 20020: 5,
 262071: 7,
 23187: 7,
 143234: 8,
 143350: 8,
 25174: 7,
 156795: 8,
 17813: 3,
 76164:

In [19]:
# printing some fsa scores
count = 0

for recipe_id in recipe_fsa:
  print(recipe_id_name[recipe_id], ":", recipe_fsa[recipe_id])

  count += 1
  if count == 10:
    break

Pork Loin, Apples, and Sauerkraut : 6
Foolproof Rosemary Chicken Wings : 7
Chicken Pesto Paninis : 7
Potato Bacon Pizza : 5
Latin-Inspired Spicy Cream Chicken Stew : 6
Reuben Sandwich I : 7
Turkey Black Bean Burgers : 5
Cranberry Pork Chops II : 7
Schnitzel Sandwich : 8
Pam's Bierocks : 6


# extracting the ingridients from the recipe

In [20]:
# nltk setup
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tarunvisvar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tarunvisvar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tarunvisvar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/tarunvisvar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
def recipe_id_to_ingredients():

  recipe_df = pd.read_csv('core-data_recipe.csv')
  recipe_id_ingredients = dict()

  lemm = WordNetLemmatizer()

  for index, row in recipe_df.iterrows():
    recipe_id = int(row['recipe_id'])
    ingredient_list = row['ingredients'].split('^')

    processed_ingredients = []

    for ingredient in ingredient_list:
      tokenized_words = word_tokenize(ingredient)
      
      lemmatized_words = []

      for word in tokenized_words:
        lemmatized_words.append(lemm.lemmatize(word))

      ingredient_words = []
      pos_tagged = nltk.pos_tag(lemmatized_words)

      for word, tag in pos_tagged:
        if tag not in ('NN', 'NNP', 'NNS', 'NNPS') or word in ('water', 'degrees', 'F', 'C', '%', 'Addition', 'Additions', '-less-sodium', '-', '*', 'Additional', 'degree', 'Rub'):
          continue
        elif tag in ('NNS', 'NNPS'):
          ingredient_words.append(word[:-1])
        else:
          ingredient_words.append(word)
      
      temp_ingredient = " ".join(ingredient_words)
      if temp_ingredient != "":
        processed_ingredients.append(temp_ingredient)

    if len(processed_ingredients) > 0:
      recipe_id_ingredients[recipe_id] = list(sorted(set(processed_ingredients)))

  return recipe_id_ingredients

In [22]:
recipe_id_ingredients = recipe_id_to_ingredients()

In [23]:
count = 0

for recipe_id in recipe_id_ingredients:
  print(recipe_id_name[recipe_id], ":", recipe_id_ingredients[recipe_id])

  count += 1
  if count == 10:
    break

Pork Loin, Apples, and Sauerkraut : ['Granny Smith apple', 'apple cider', 'boneless pork loin roast', 'brown sugar', 'caraway seed', 'ground pepper', 'onion', 'powder', 'salt', 'sauerkraut']
Foolproof Rosemary Chicken Wings : ['chicken wing', 'head garlic', 'oil', 'pepper', 'salt', 'sprig rosemary']
Chicken Pesto Paninis : ['Monterey Jack cheese', 'basil pesto', 'bell pepper', 'bread', 'chicken', 'onion']
Potato Bacon Pizza : ['Crust', 'Parmesan cheese', 'Sauce', 'butter', 'flour', 'honey', 'mozzarella cheese', 'oil', 'potato', 'strip bacon', 'whipping cream', 'yeast']
Latin-Inspired Spicy Cream Chicken Stew : ['bean', 'cilantro', 'cream cheese', 'ground cumin', 'ground pepper', 'kerne corn', 'salsa', 'skinless boneless breast half', 'taco', 'tomato']
Reuben Sandwich I : ['beef', 'butter', 'mozzarella cheese', 'rye bread', 'sauerkraut']
Turkey Black Bean Burgers : ['bean', 'egg', 'garlic', 'ground cumin', 'ground turkey', 'onion', 'oregano', 'parsley', 'sea salt ground pepper', 'tomato

In [24]:
total_ingredients_list = []
for ingredients in recipe_id_ingredients.values():
  for ingredient in ingredients:
    total_ingredients_list.append(ingredient)

total_ingredients_set = list(sorted(set(total_ingredients_list)))
print(len(total_ingredients_set))

14744


# Calculating pmi score

In [25]:
edges = []
#Each edge is an edge connecting recipe_id's to their corresponding ingredients
for recipe_id in recipe_id_ingredients:
  for ingredient in recipe_id_ingredients[recipe_id]:
    edges.append((recipe_id, ingredient))

In [26]:
import networkx as nx
from networkx.algorithms import bipartite

In [27]:
recipe_df = pd.read_csv('core-data_recipe.csv')
#Creating a bipartite graph with recipe_is's on one side and ingredients on other side
B = nx.Graph()
#Adding nodes for left side
B.add_nodes_from(list(recipe_df['recipe_id']), bipartite = 0)
#Adding nodes for right side
B.add_nodes_from(total_ingredients_set, bipartite = 1)
#Adding edges
B.add_edges_from(edges)

In [28]:
#Creating a projection of ingredients with themselves(helps in finding pmi score)
G = bipartite.weighted_projected_graph(B, total_ingredients_set)

In [29]:
adjacency_matrix = G.adj

In [30]:
def calculate_pmi_ingredient(ingredient):
  s = 0
  #Calculate how many ingredients an ingredient co occurs with based on the projection of bipartite graph
  for key in adjacency_matrix[ingredient]:
    s += adjacency_matrix[ingredient][key]['weight']

  return s

In [31]:
ingredient_pmi = dict()
for ingredient in total_ingredients_set:
  ingredient_pmi[ingredient] = calculate_pmi_ingredient(ingredient)

In [32]:
ingredient_pmi

{'APPLE': 12,
 'APPLE CIDER SYRUP': 14,
 'APPLE FILLING': 12,
 'APPLE FLLING': 13,
 'ATHENOS Traditional Crumbled Feta Cheese': 6,
 'Accents® Organic Sprouted Quinoa Trio': 7,
 'Acesulfame Potassium sweetener': 11,
 'Adobo': 4,
 'Adzuki Bean Filling': 10,
 'Agave Nectar': 6,
 'Aioli': 30,
 'Alaskan salmon': 4,
 'Aleppo pepper': 58,
 'Aleppo pepper flake': 35,
 'Alfredo Sauce': 69,
 'Alfredo pasta sauce': 36,
 'Alfredo sauce': 473,
 'Alfredo sauce Classico®': 34,
 'Alfredo sauce egg mix': 6,
 'Alfredo sauce mix': 7,
 'Alfredo sauce mix Knorr®': 10,
 'AllWhites® egg': 8,
 'Almond Breeze Almondmilk Vanilla': 13,
 'Almond Breeze Original OR Almond Breeze Almond-Cashew Blend': 20,
 'Almond Butter Swirl': 11,
 'Almond Filling': 14,
 'Almond Meringue Cookie': 10,
 'Aluminum foil': 28,
 'Amaretto liqueur': 4,
 'Amarula liqueur': 3,
 'American': 37,
 'American Cheddar blend': 20,
 'American Cheese half': 3,
 'American cheese': 701,
 'American cheese slice': 12,
 'Anaheim': 11,
 'Anaheim New Mex

In [33]:
from math import log

#Calculates pmi for a pair of ingredients based on the formula mentioned in paper
def calculate_pmi_pair(a, b):
  try:
    pab = adjacency_matrix[a][b]['weight']
  except:
    return 0

  pa = ingredient_pmi[a]
  pb = ingredient_pmi[b]

  return log((pab * len(list(recipe_df['recipe_id']))) / (pa * pb))

In [34]:
def get_top_n_pmi_ingredients(ingredient, n):
  pmi_ls = [(ing,calculate_pmi_pair(ingredient,ing)) for ing, wt in adjacency_matrix[ingredient].items()]
  pmi_ls.sort(key=lambda x:x[1], reverse=True)
  return pmi_ls[:n]

get_top_n_pmi_ingredients('oil',10)

[('bagel cut slice', -1.5766738033575045),
 ('cashew Stock Barrel®', -1.5766738033575045),
 ('ciabatt cube', -1.5766738033575045),
 ('coconut curry sauce KanTong®', -1.5766738033575045),
 ('corn meal P.A.N.®', -1.5766738033575045),
 ('durum wheat flour', -1.5766738033575045),
 ('plantain thick', -1.5766738033575045),
 ('steak spice mix', -1.5766738033575045),
 ('zucchini thick lengthwise', -1.5766738033575045),
 ('tofu plain', -1.5766738033575045)]

## Cells after this till the next markdown are for other purposes.

In [35]:
def calculate_pmi_recipe(recipe_id):
  ingredients = recipe_id_ingredients[recipe_id]

  if len(ingredients) < 2:
    return 0

  max_pmi_score = 0
    #Calculate pmi for each pair of ingredients in a recipe
  for i in range(len(ingredients)):
    for j in range(i+1, len(ingredients)):
      pmi = calculate_pmi_pair(ingredients[i], ingredients[j])

      if pmi > max_pmi_score:
        max_pmi_score = pmi
  #Returns the score of pair with maximum pmi score in the recipe
  return max_pmi_score

In [36]:
recipe_id_pmi = dict()
for recipe_id in recipe_id_ingredients:
  recipe_id_pmi[recipe_id] = calculate_pmi_recipe(recipe_id)

#This dictionary thus contains just the maximum pmi for each recipe.
#We are not using this currently as pmi for a 'recipe' is not used. 
#We just compute pmi of ingredient pair for one ing with all other ings later in the notebook

KeyboardInterrupt: 

In [ ]:
import pickle

with open("recipe_id_pmi.bin","wb") as file:
  pickle.dump(recipe_id_pmi, file)  

In [ ]:
with open("recipe_id_pmi.bin","rb") as file:
  recipe_id_pmi = pickle.load(file)

In [ ]:
# normalizing the pmi score
def normalise_pmi_scores(recipe_id_pmi, upper_bound = 5):
    #Just performing min max normalization of pmi score of a recipe with respect to all recipes.
  pmi_values = list(recipe_id_pmi.values())
  
  mn = min(pmi_values)
  mx = max(pmi_values)

  for recipe_id in recipe_id_pmi:
    recipe_id_pmi[recipe_id] = ((recipe_id_pmi[recipe_id] - mn)/(mx - mn)) * upper_bound
  return recipe_id_pmi

In [ ]:
recipe_id_pmi = normalise_pmi_scores(recipe_id_pmi)

In [46]:
count = 0

for recipe_id in recipe_id_pmi:
  print(recipe_id_name[recipe_id], ':', recipe_id_pmi[recipe_id])

  count += 1
  if count == 10:
    break

Pork Loin, Apples, and Sauerkraut : 0
Foolproof Rosemary Chicken Wings : 0
Chicken Pesto Paninis : 0
Potato Bacon Pizza : 0
Latin-Inspired Spicy Cream Chicken Stew : 0
Reuben Sandwich I : 1.1220062455596929
Turkey Black Bean Burgers : 0
Cranberry Pork Chops II : 0
Schnitzel Sandwich : 3.5429336583410316
Pam's Bierocks : 0


# Creating model which can predict the fsa score of any set of ingredients

In [47]:
import numpy as np
enc_ingredients_list = list(total_ingredients_set)

def encode_ingredients(ingredients):
    #Converting ingredients to vectors to convert it into the format to pass into MLP model.
  res = np.array(enc_ingredients_list)
  f = np.vectorize(lambda x: int(x in ingredients))
  return f(res)

In [48]:
# def encode_ingredients(ingredients):
#   res = []
#   for i in total_ingredients_list:
#     if i in ingredients:
#       res.append(1)
#     else:
#       res.append(0)
#   return res

In [49]:
# using MLP for the prediction
# the output will be the predicted fsa score 
# input will be one hot encoded vector representing the ingredients used

# recipe_id_ingredients
# recipe_fsa 

# Creating the dataset of MLP
X, y = [],[]
count = 0
for recipe_id in recipe_id_ingredients:
  X.append(encode_ingredients(recipe_id_ingredients[recipe_id]))
  y.append(recipe_fsa[recipe_id])
  count += 1
  if count == 1000:
    break


In [50]:
from sklearn.neural_network import MLPRegressor
#Using the MLP from sklearn library
regr = MLPRegressor((100,50,30,))
regr.fit(X, y)

MLPRegressor(hidden_layer_sizes=(100, 50, 30))

In [51]:
regr.score(X[10:30],y[10:30])

0.9992859142453059

# Combining the fsa, pmi to recommend healthy recipe in the recommendation

In [52]:
def get_alternate_ingredients(ingredients):
  used_ing_set = set(ingredients)
    #used_ing_set : ingredients already used in this recipe
  min_occur = float('inf')
  min_ing = None
  for ingredient in ingredients:
        ing_pmi = calculate_pmi_ingredient(ingredient)
        if ing_pmi<min_occur:
            min_occur = ing_pmi
            min_ing = ingredient
            
  ing = min_ing
  local_pmi_ls = []

  for i in ingredients:
    if i == ing:
      continue
    local_pmi_ls.append((calculate_pmi_pair(ing,i), i))

    #Sorting based on highest pmi of this ing with others 
  local_pmi_ls.sort(key=lambda x:x[0], reverse=True)
  
  anchor = local_pmi_ls[0][1] # Using that maximum pmi ingredient as anchor find the replacement for ing
  anchor_pmi_ls = get_top_n_pmi_ingredients(anchor, 20) #Top 20 cooccurences of the anchor ing in the graph
  
  last_used = ing
  ing_fsa = []

  for alternates, a_pmi in anchor_pmi_ls:
    if alternates not in used_ing_set:
        #Adding alternate ing to used ingredients in this recipe
      used_ing_set.add(alternates)
        #Remove existing ing(the one to be replaced)
      ingredients.remove(last_used)
        #Adding the new alternative 
      ingredients.append(alternates)
        #Converting thsi to be last used so that it can be replaced again for next iteration
      last_used = alternates
        #Now trying out the replaced alternative for new fsa score and using this alternative only if fsa is better
      n_fsa = regr.predict(np.array([encode_ingredients(ingredients)]))
      ing_fsa.append((n_fsa[0], ingredients.copy()))
  
  ing_fsa.sort(key=lambda x:x[0], reverse=True)
  return ing_fsa[0] # returns (fsa, ingredient list)

In [53]:
# top_n_predictions -> (userid, [(itemid, est),...])
import random

def get_healthy_recommendations(recommendation):
  res_recipe_ingredients = []
  for recipe,est in recommendation:
    #Calculating(retrieving from previously calculated) fsa of the recipe recommended by SVD to user
    ingredients = recipe_id_ingredients[recipe]
    fsa = recipe_fsa[recipe]

    if fsa > 6:
        #If the fsa>6, it is into red zone
      alt_fsa, alternate_ingredients = get_alternate_ingredients(ingredients)
        #Calling get_alternate)_ingredients, that would try replacing a random ingredient after choosing an anchor.
        #It would try many replacements and predict the fsa for each replacement and return that list of ingredients with best predicted fsa after trying all replacements(alternative ings)
      if alt_fsa < fsa:
        #If the best alternate fsa is below red zone, we replac that ingredient in the recipe
        print(f'replacing {ingredients} with new ingredients {alternate_ingredients}')
        print(f'Old fsa  = {fsa}')
        print(f'New fsa = {alt_fsa}')
        print('Recipe ',recipe_id_name[recipe])
        ingredients = alternate_ingredients
    #Creating a list of novel recipes (recipes with ingredients replaced on SVD recommended recipes)
    res_recipe_ingredients.append((recipe, ingredients))
    #List of recipes and their alternated ingredients
  return res_recipe_ingredients

In [54]:
uid = 481224
#Normal Top n recipes recommendation for a user acc. to collaborative filtering using SVD
#'top_n_predictions' is a dictionary that contaisn top nrecipe recommendations for a user precomputed(scroll aboev the notebook, did long back)
#So that, it can be accessed easily.
#It is a dictionary containing keys as user_ids and values as the list of size(top_n) containing tuples of (item_id,estimated rating given by user for the predicted recipe)
#Now, we are trying to recommend for the user uid = 481224
recommendation = top_n_predictions[uid]
recommendation #(recipe_id, user's predicted rating for that recipe)

healthy_recommendations = get_healthy_recommendations(recommendation)
healthy_recommendations

replacing ['brown sugar', 'butter', 'confectioner sugar', 'cream cheese', 'egg', 'flour', 'ground cinnamon', 'milk', 'salt', 'sugar', 'vanilla extract', 'sauce Valentina'] with new ingredients ['brown sugar', 'butter', 'confectioner sugar', 'cream cheese', 'egg', 'flour', 'ground cinnamon', 'milk', 'salt', 'sugar', 'vanilla extract', 'matzo']
Old fsa  = 8
New fsa = 5.956813519674859
Recipe  Cinnamon Rolls II


[(42719,
  ['banan', 'butter', 'egg', 'flour', 'powder', 'salt', 'soda', 'sugar']),
 (50644,
  ['Parmesan cheese',
   'basil',
   'butter',
   'celery salt',
   'ground pepper',
   'lemon juice',
   'mayonnaise',
   'onion powder',
   'tilapia fillet']),
 (24351,
  ['brown sugar',
   'butter',
   'confectioner sugar',
   'cream cheese',
   'egg',
   'flour',
   'ground cinnamon',
   'milk',
   'salt',
   'sugar',
   'vanilla extract',
   'matzo'])]

In [55]:
print(f'user {uid} recommendations : ')
for recipe_id, ings in healthy_recommendations:
  print(f"{recipe_id_name[recipe_id]} with ingredients {ings}")

user 481224 recommendations : 
Banana Muffins II with ingredients ['banan', 'butter', 'egg', 'flour', 'powder', 'salt', 'soda', 'sugar']
Broiled Tilapia Parmesan with ingredients ['Parmesan cheese', 'basil', 'butter', 'celery salt', 'ground pepper', 'lemon juice', 'mayonnaise', 'onion powder', 'tilapia fillet']
Cinnamon Rolls II with ingredients ['brown sugar', 'butter', 'confectioner sugar', 'cream cheese', 'egg', 'flour', 'ground cinnamon', 'milk', 'salt', 'sugar', 'vanilla extract', 'matzo']
